# Simulation results analysis

This is a notebook to use some of the functions written in results_processing.py 
to analyse the results of simulations and produce plots.

First, let's import the data and extract total energy usage per floor.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datamethods.results_processing import *
import warnings
warnings.filterwarnings('ignore')

# Find what files relate to which built islands
d = get_building_files("model_idfs_rerun_2")

# Read in simstock data
df = pd.read_csv("preprocessed_croydon.csv")
gdf = gpd.GeoDataFrame(
    df.loc[:, [c for c in df.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
    crs="epsg:27700",
)
gdf = gdf.drop(columns=["Unnamed: 0"])

# Add energy use columns
for i in range(1,21):
    gdf[f"FLOOR_{i}: total_elec_energy"] = None
    gdf[f"FLOOR_{i}: total_hvac_energy"] = None
    gdf[f"FLOOR_{i}: mean_annual_temperature"] = None
    gdf[f"FLOOR_{i}: time_above_26"] = None
    gdf[f"FLOOR_{i}: time_below_16"] = None
    gdf[f"FLOOR_{i}: max_temp"] = None
    gdf[f"FLOOR_{i}: min_temp"] = None
gdf["site_total_elec_energy"] = None
gdf["site_total_elec_energy_per_floor_area"] = None
gdf["site_total_hvac_energy"] = None
gdf["site_total_hvac_energy_per_floor_area"] = None
gdf["site_mean_temperature"] = None
gdf["site_mean_time_above_26"] = None
gdf["site_mean_time_below_16"] = None
gdf["site_mean_max_temp"] = None
gdf["site_mean_min_temp"] = None

# Iterate over the rows
for i in range(len(gdf)):
    print(f'\rProcessing row {i+1} of {len(gdf)}', end='', flush=True)

    # Get the scu and bi number
    scu = gdf.iloc[i]["osgb"]
    bi_num = gdf.iloc[i]["bi"]

    # Get the relevent file number
    out_dir_num = d[bi_num]
    out_dir = f"model_idfs_rerun_2/built_island_{out_dir_num}_ep_outputs"

    # Get the time series for this scu
    ts_dict = get_scus_elec_timeseries(scu, out_dir)
    ts_dict_hvac = get_scus_hvac_timeseries(scu, out_dir)
    ts_dict_temp = get_temperature_timeseries(scu, out_dir)

    # Iterate over floor numbers
    floor_sum = 0
    mean_temp_sum = 0
    time_above_sum = 0
    time_below_sum = 0
    max_temp_sum = 0
    min_temp_sum = 0
    floor_sum_hvac = 0
    active_floors = 0
    for floor_num in ts_dict.keys():
        tot = ts_dict[floor_num].total()
        tot_hvac = ts_dict_hvac[floor_num].total()
        mean_temp = ts_dict_temp[floor_num].mean()
        time_above = ts_dict_temp[floor_num].hours_above_max()
        time_below = ts_dict_temp[floor_num].hours_below_min()
        max_temp = ts_dict_temp[floor_num].max()
        min_temp = ts_dict_temp[floor_num].min()

        # Place into the dataframe
        gdf.at[i, f"FLOOR_{floor_num}: total_elec_energy"] = tot
        gdf.at[i, f"FLOOR_{floor_num}: total_hvac_energy"] = tot_hvac
        gdf.at[i, f"FLOOR_{floor_num}: mean_annual_temperature"] = mean_temp
        gdf.at[i, f"FLOOR_{floor_num}: time_above_26"] = time_above/8760
        gdf.at[i, f"FLOOR_{floor_num}: time_below_16"] = time_below/8760
        gdf.at[i, f"FLOOR_{floor_num}: max_temp"] = max_temp
        gdf.at[i, f"FLOOR_{floor_num}: min_temp"] = min_temp
        
        # Tally up the energy and temperatures over all the floors
        floor_sum += tot
        floor_sum_hvac += tot_hvac
        active_floors += 1
        mean_temp_sum += mean_temp
        time_above_sum += time_above
        time_below_sum += time_below
        max_temp_sum += max_temp
        min_temp_sum += min_temp

    gdf.at[i, "site_total_elec_energy"] = floor_sum
    gdf.at[i, "site_total_elec_energy_per_floor_area"] = floor_sum/(active_floors*gdf.iloc[i]["geometry"].area)

    gdf.at[i, "site_total_hvac_energy"] = floor_sum_hvac
    gdf.at[i, "site_total_hvac_energy_per_floor_area"] = floor_sum_hvac/(active_floors*gdf.iloc[i]["geometry"].area)

    gdf.at[i, "site_mean_temperature"] = mean_temp_sum/active_floors
    gdf.at[i, "site_mean_time_above_26"] = time_above_sum/(active_floors*8760)
    gdf.at[i, "site_mean_time_below_16"] = time_below_sum/(active_floors*8760)
    gdf.at[i, "site_mean_max_temp"] = max_temp_sum/active_floors
    gdf.at[i, "site_mean_min_temp"] = min_temp_sum/active_floors

Now let's plot some results. First, the overheating on a floor-by-floor basis.

In [ ]:
# Make a plotting function
from typing import Union
from matplotlib.axes._axes import Axes
from matplotlib.collections import PatchCollection
from shapely.geometry import Polygon, box
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import contextily as cx
import xyzservices.providers as xyz
from matplotlib.pyplot import Normalize
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


def plot_geometry(geom: Polygon, ax: Axes, **kwargs) -> PatchCollection:
    """
    Function to plot shapely polygons.
    """
    path = Path.make_compound_path(
        Path(np.asarray(geom.exterior.coords)[:, :2]),
        *[Path(np.asarray(ring.coords)[:, :2]) for ring in geom.interiors])
    patch = PathPatch(path, **kwargs)
    collection = PatchCollection([patch], **kwargs)
    ax.add_collection(collection, autolim=True)
    ax.autoscale_view()
    return collection

def plot_buildings(gdf: gpd.GeoDataFrame, colour_variable: str, background: bool = True, ax: Union[Axes, None] = None, vmax: float = 1.0) -> None:
    """
    Function to plot the region on a background map.
    """
    # Set up the plot
    if not ax:
        _, ax = plt.subplots(1, 1)

    # Make the colourmap
    norm = plt.Normalize(0.0, vmax)
    cmap = plt.cm.get_cmap("Reds")
    scalar_mappable = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
    scalar_mappable.set_array([])

    for _, row in gdf.iterrows():
        if row[colour_variable]:
            plot_geometry(row["geometry"], ax, color=scalar_mappable.to_rgba(row[colour_variable]), edgecolor="k", linewidth=0.4)

    
    # Add an open street map background if requested
    if background:
        cx.add_basemap(ax, alpha=0.6, crs="epsg:27700", source=cx.providers.OpenStreetMap.Mapnik)

    plt.xticks([])
    plt.yticks([])
    plt.colorbar(scalar_mappable, label=colour_variable, ax=ax)

# Now let's use these functions.
# Plotting the GeoDataFrame with total_energy column
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Calculate the minimal bounding box (envelope) around all geometries
envelope = gdf.envelope
bbox = envelope.unary_union

# Define the coordinates for the lower right quadrant bounding box
lower_right_bbox = box(bbox.bounds[2], bbox.bounds[1], bbox.centroid.x, bbox.centroid.y)

# Filter the original GeoDataFrame to get only the geometries in the lower left quadrant
lower_right_gdf = gdf[gdf.geometry.within(lower_right_bbox)]

# Get the bounding box of the lower_left_gdf
minx, miny, maxx, maxy = lower_right_gdf.total_bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

# Iterate over the floors and plot them
plot_buildings(lower_right_gdf, "FLOOR_1: time_above_26", ax=ax, vmax=0.6)
for i in range(2,10):

    # Plotting the GeoDataFrame with total_energy column
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))

    # Get the bounding box of the lower_left_gdf
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    plot_buildings(lower_right_gdf, f"FLOOR_{i}: time_above_26", ax=ax, vmax=0.6, background=True)

Now let's plot overheating per as a afunction of usage catagory.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Collate data by usage catagory
data = {
    "Office": []
}
for ind, row in gdf.iterrows():
    for i in range(1,20):
        floor_use = row[f"FLOOR_{i}: use"]
        if floor_use:
            if type(floor_use) == str:
                if floor_use not in data.keys():
                    data[floor_use] = [row[f"FLOOR_{i}: time_above_26"]]
                else:
                    data[floor_use].append(row[f"FLOOR_{i}: time_above_26"])

categories_to_plot = ["Factory", "Community", "Education", "Sport", "Emergency", "Warehouse", "Dwell"]
selected_data = {category: data[category] for category in categories_to_plot}

# Extract category names and values
categories = list(selected_data.keys())
values = list(selected_data.values())

# Create box plots
plt.boxplot(values, labels=categories)
plt.ylabel('Percentage of time over 26')
plt.tight_layout()
plt.show()

Now plot total site energy usage per floor area.

In [ ]:
import contextily as cx
import xyzservices.providers as xyz
from matplotlib.pyplot import Normalize
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Plotting the GeoDataFrame with total_energy column
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

gdf["total_energy"] = gdf["site_total_hvac_energy"] + gdf["site_total_elec_energy"]
plot = gdf.plot(column='site_total_hvac_energy_per_floor_area', cmap='viridis', linewidth=0.4, ax=ax, edgecolor='0')
cx.add_basemap(ax, crs=gdf.crs.to_string(), alpha=0.6, source=cx.providers.OpenStreetMap.Mapnik)


# Create a mappable
norm = Normalize(vmin=gdf['site_total_hvac_energy_per_floor_area'].min(), vmax=gdf['site_total_hvac_energy_per_floor_area'].max())
sm = plt.cm.ScalarMappable(cmap='viridis', norm=norm)
sm.set_array([])

# Adding a colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.1)
cbar = plt.colorbar(sm, cax=cax)
cbar.set_label('Total Energy')


ax.set_axis_off()
plt.show()



Now plot the usage plot.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot = gdf.plot(column='type', linewidth=0.4, ax=ax, edgecolor='0', legend=True)
ax.set_axis_off()
cx.add_basemap(ax, crs=gdf.crs.to_string(), alpha=0.6, source=cx.providers.OpenStreetMap.Mapnik)
plt.show()

Now let's do the time series of energy usage.

In [ ]:
# Start 3 empty time series: one each for mixed, dom, and non-dom
mixed_ts = TimeSeries()
dom_ts = TimeSeries()
nondom_ts = TimeSeries()

# For row in gdf
for i in range(len(gdf)):
    print(f'\rProcessing row {i} of {len(gdf)}', end='', flush=True)
    
    # Get the row's time series
    # Get the scu and bi number
    scu = gdf.iloc[i]["osgb"]
    bi_num = gdf.iloc[i]["bi"]

    # Get the relevent file number
    out_dir_num = d[bi_num]
    out_dir = f"model_idfs_rerun_2/built_island_{out_dir_num}_ep_outputs"

    # Get the time series for this scu
    ts_dict = get_scus_elec_timeseries(scu, out_dir)
    ts_dict_hvac = get_scus_hvac_timeseries(scu, out_dir)
    
    # Sum the floors in the time series 
    # to get a total time series for this house
    site_ts = TimeSeries()
    for floor_num in ts_dict.keys():
        site_ts = add_timeseries(site_ts, ts_dict[floor_num])
        site_ts = add_timeseries(site_ts, ts_dict_hvac[floor_num])
    
    # Add it to one of the running time series 
    # according to type
    if gdf.iloc[i]["type"] == "dom":
        dom_ts = add_timeseries(dom_ts, site_ts)
    elif gdf.iloc[i]["type"] == "nondom":
        nondom_ts = add_timeseries(nondom_ts, site_ts)
    else:
        mixed_ts = add_timeseries(mixed_ts, site_ts)

dom_weekly = []
runsum = 0
for i in range(len(nondom_ts.x)):
    if i % (24*7) == 0:
        dom_weekly.append(runsum)
        runsum = 0
    runsum += dom_ts.x[i]

nondom_weekly = []
runsum = 0
for i in range(len(nondom_ts.x)):
    if i % (24*7) == 0:
        nondom_weekly.append(runsum)
        runsum = 0
    runsum += nondom_ts.x[i]

mixed_weekly = []
runsum = 0
for i in range(len(nondom_ts.x)):
    if i % (24*7) == 0:
        mixed_weekly.append(runsum)
        runsum = 0
    runsum += mixed_ts.x[i]

def cshift(lst: list) -> list:
    return lst[len(lst)//2:] + lst[:len(lst)//2]
weeklyts = dom_ts.weekly_aggregates().index
y = [cshift(dom_weekly[1:]), cshift(mixed_weekly[1:]), cshift(nondom_weekly[1:])]

# Create a stacked area plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.stackplot(weeklyts[1:], y, labels=['d', 'm', 'nd'], alpha=0.7)

# Customize the plot
ax.set_xlim([weeklyts[1], weeklyts[-1]])
ax.set_xticks([weeklyts[1], weeklyts[-1]])
ax.set_ylim([0,2.5e6])
ax.legend()
plt.show()

Now plot the stacked week in Jan electricity usage panel for whole area, and then we get into the disaggregated panel plot.

In [ ]:
# Start 3 empty time series: one each for mixed, dom, and non-dom
mixed_ts = TimeSeries()
dom_ts = TimeSeries()
nondom_ts = TimeSeries()

# For row in gdf
for i in range(len(gdf)):
    print(f'\rProcessing row {i} of {len(gdf)}', end='', flush=True)
    
    # Get the row's time series
    # Get the scu and bi number
    scu = gdf.iloc[i]["osgb"]
    bi_num = gdf.iloc[i]["bi"]

    # Get the relevent file number
    out_dir_num = d[bi_num]
    out_dir = f"model_idfs_rerun_2/built_island_{out_dir_num}_ep_outputs"

    # Get the elec time series for this scu
    ts_dict = get_scus_elec_timeseries(scu, out_dir)
    
    # Sum the floors in the time series 
    # to get a total time series for this house
    site_ts = TimeSeries()
    for floor_num in ts_dict.keys():
        site_ts = add_timeseries(site_ts, ts_dict[floor_num])
    
    # Add it to one of the running time series 
    # according to type
    if gdf.iloc[i]["type"] == "dom":
        dom_ts = add_timeseries(dom_ts, site_ts)
    elif gdf.iloc[i]["type"] == "nondom":
        nondom_ts = add_timeseries(nondom_ts, site_ts)
    else:
        mixed_ts = add_timeseries(mixed_ts, site_ts)

# Now plot a the first week in January
dom_week = dom_ts.x[1:24*7]
nondom_week = nondom_ts.x[1:24*7]
mixed_week = mixed_ts.x[1:24*7]
y = [dom_week, mixed_week, nondom_week]

# Create a stacked area plot
fig, ax = plt.subplots(figsize=(8, 6))
ax.stackplot(dom_ts.t[1:24*7], y, labels=['d', 'm', 'nd'], alpha=0.7)

# Customize the plot
ax.set_xlim([dom_ts.t[1:24*7][1], dom_ts.t[1:24*7][-1]])
ax.set_xticks([dom_ts.t[1:24*7][1], dom_ts.t[1:24*7][-1]])
ax.set_ylim([0,18000])
ax.legend()
plt.show()

Now let's get a zoomed in version, with floors disaggregated. First, let's soom into a region with high mixing.

In [ ]:
from shapely.geometry import box

# Calculate the minimal bounding box (envelope) around all geometries
envelope = gdf.envelope
bbox = envelope.unary_union

# Define the coordinates for the lower right quadrant bounding box
lower_right_bbox = box(bbox.bounds[2], bbox.bounds[1], bbox.centroid.x, bbox.centroid.y)

# Filter the original GeoDataFrame to get only the geometries in the lower left quadrant
lower_right_gdf = gdf[gdf.geometry.within(lower_right_bbox)]

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot = lower_right_gdf.plot(column='type', linewidth=0.4, ax=ax, edgecolor='0', legend=True)
ax.set_axis_off()
cx.add_basemap(ax, crs=gdf.crs.to_string(), alpha=0.6, source=cx.providers.OpenStreetMap.Mapnik)
plt.show()

Now plot each layer.

In [ ]:
# Iterate over the floor columns and find the global minimum and maximum values
global_min_floor = float('inf')
global_max_floor = float('-inf')
for j in range(1,12):
    min_floor_j = lower_right_gdf[f'FLOOR_{j}: total_elec_energy'].min()
    max_floor_j = lower_right_gdf[f'FLOOR_{j}: total_elec_energy'].max()

    # Update global minimum and maximum values
    global_min_floor = min(global_min_floor, min_floor_j)
    global_max_floor = max(global_max_floor, max_floor_j)

print(global_max_floor)
print(global_min_floor)

# Plot the first floor with a background
# Plotting the GeoDataFrame with total_energy column
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

# Choose the floor index i (e.g., 1 for floor_1, 2 for floor_2, etc.)
i = 1


# Get the bounding box of the lower_left_gdf
minx, miny, maxx, maxy = lower_right_gdf.total_bounds

# Filter out buildings with empty floor_i
filtered_gdf = lower_right_gdf[lower_right_gdf[f'FLOOR_{i}: total_elec_energy'].notnull()]

# filtered_gdf.to_csv("filtered.csv")

plot = filtered_gdf.plot(column=f'FLOOR_{i}: total_elec_energy', cmap='viridis', linewidth=0.4, ax=ax, edgecolor='0')
cx.add_basemap(ax, crs=filtered_gdf.crs.to_string(), alpha=0.6, source=cx.providers.OpenStreetMap.Mapnik)


ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

# Show the plot
ax.set_xticks([])
ax.set_yticks([])

# Now flot the remaining floors with no background
for i in range(2,7):

    # Plotting the GeoDataFrame with total_energy column
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))

    # Filter out buildings with empty floor_i
    filtered_gdf = lower_right_gdf[lower_right_gdf[f'FLOOR_{i}: total_elec_energy'].notnull()]

    plot = filtered_gdf.plot(column=f'FLOOR_{i}: total_elec_energy', cmap='viridis', linewidth=0.4, ax=ax, edgecolor='0')

    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # Show the plot
    ax.set_xticks([])
    ax.set_yticks([])

Now focus on a specific building and its mixing.

In [ ]:
# Filter the DataFrame to get the specific row
target_row = gdf[gdf['osgb'] == 52401021810002]
building_geometry = target_row.geometry.iloc[0]

# Specify the distance (in meters) to expand the bounding box
expansion_distance = 10

# Calculate the expanded bounding box
expanded_building = building_geometry.buffer(expansion_distance)

# Plot the specific building with an expanded area and a basemap
ax = gdf.plot(figsize=(10, 10), color="b", edgecolor="k")  # Plot all buildings in the background
target_row.plot(ax=ax, color='r', edgecolor='black')  # Plot the specific building in red
# cx.add_basemap(ax, crs=gdf.crs.to_string(), alpha=0.3)

# Set the plot extent to include the expanded area
ax.set_xlim(expanded_building.bounds[0], expanded_building.bounds[2])
ax.set_ylim(expanded_building.bounds[1], expanded_building.bounds[3])

# Show the plot
ax.set_xticks([])
ax.set_yticks([])
# plt.savefig(f"paperfigs/mixed_floor_2.pdf")
plt.show()

Plot this building's timeseries.

In [ ]:
# Get the row's time series
scu = 52401021810002
bi_num = gdf[gdf["osgb"] == scu]["bi"].values[0]

# Get the relevent file number
out_dir_num = d[bi_num]
out_dir = f"model_idfs_rerun_2/built_island_{out_dir_num}_ep_outputs"

# Get the time series for this scu
ts_dict = get_scus_elec_timeseries(scu, out_dir)
ts_dict_hvac = get_scus_hvac_timeseries(scu, out_dir)

# Extract the timeseries
shop_elec = cshift(ts_dict[1].x)
shop_hvac = cshift(ts_dict_hvac[1].x)
dwell_elec = cshift(ts_dict[2].x)
dwell_hvac = cshift(ts_dict_hvac[2].x)

shop_elec_daily, shop_hvac_daily = [], [] 
dwell_elec_daily, dwell_hvac_daily = [], []
shop_elec_sum, shop_hvac_sum = 0, 0
dwell_elec_sum, dwell_hvac_sum = 0, 0
ts = []
for i in range(len(shop_elec)):
    if i % 24 == 0:
        ts.append(ts_dict[1].t[i])
        shop_elec_daily.append(shop_elec_sum)
        shop_hvac_daily.append(shop_hvac_sum)
        dwell_elec_daily.append(dwell_elec_sum)
        dwell_hvac_daily.append(dwell_hvac_sum)
        shop_elec_sum = 0
        shop_hvac_sum = 0
        dwell_elec_sum = 0
        dwell_hvac_sum = 0
    shop_elec_sum += shop_elec[i]
    shop_hvac_sum += shop_hvac[i]
    dwell_elec_sum += dwell_elec[i]
    dwell_hvac_sum += dwell_hvac[i]

# Normalise
for i in range(1,365):
    total = shop_elec_daily[i] + shop_hvac_daily[i] + dwell_elec_daily[i] + dwell_hvac_daily[i]
    shop_elec_daily[i] = 100*shop_elec_daily[i]/total
    shop_hvac_daily[i] = 100*shop_hvac_daily[i]/total
    dwell_elec_daily[i] = 100*dwell_elec_daily[i]/total
    dwell_hvac_daily[i] = 100*dwell_hvac_daily[i]/total

# Create a stacked area plot
fig, ax = plt.subplots(figsize=(8, 6))
y = [shop_elec_daily, shop_hvac_daily, dwell_elec_daily, dwell_hvac_daily]
ax.stackplot(ts, y, labels=['shop_e', 'shop_hv', 'dwell_e', 'dwell_hv'], alpha=0.7)

# Customize the plot
ax.set_xlim([ts[0], ts[-1]])
ax.set_xticks([ts[0], ts[-1]])
ax.set_ylim([0,100])
ax.set_yticks([0,100])
ax.legend()
plt.show()

Now lets make a horizontal bar chart of the total energy contributions of different floors by type.

In [ ]:
# Holders for sums 
dom_floor_run_sum = 0
nondom_floor_run_sum = 0
dom_building_count_run_sum = 0
nondom_building_count_run_sum = 0
mixed_building_count_run_sum = 0

# Iterate over rows
for index, row in gdf.iterrows():

    print(f"Processing index {index} of {gdf.shape[0]}!")

    # Iterate over floors
    for i in range(1,21):

        # Check if the floor is occupied
        usage = row[f"FLOOR_{i}: use"]
        if str(usage) != "nan":

            # Find the floor's total energy usage 
            floor_tot = row[f"FLOOR_{i}: total_hvac_energy"] + row[f"FLOOR_{i}: total_elec_energy"]

            # Find the floor's usage 
            if usage == "Dwell":
                dom_floor_run_sum += floor_tot
            else:
                nondom_floor_run_sum += floor_tot

    # Also get the total building contribution
    if row["type"] == "nondom":
        nondom_building_count_run_sum += row["total_energy"]
    elif row["type"] == "dom":
        dom_building_count_run_sum += row["total_energy"]
    elif row["type"] == "mixed":
        mixed_building_count_run_sum += row["total_energy"]

Now make a horizontal pie cart representing these contributions.

In [ ]:
percentages = [
                100*dom_floor_run_sum/(dom_floor_run_sum + nondom_floor_run_sum),
                100*nondom_floor_run_sum/(dom_floor_run_sum + nondom_floor_run_sum)
               ]

print(f"Total = {dom_floor_run_sum + nondom_floor_run_sum}")
print(percentages)

# Colors for each segment
colors = ['blue', 'orange']

# Calculate cumulative percentages
cumulative_percentages = [sum(percentages[:i+1]) for i in range(len(percentages))]

# Create the horizontal bar plot
fig, ax = plt.subplots()

for i in range(len(percentages)):
    width = percentages[i]
    start_percentage = cumulative_percentages[i] - width if i > 0 else 0
    rect = plt.Rectangle((start_percentage, 0), width, 1, color=colors[i], edgecolor='white')
    ax.add_patch(rect)

# Set plot limits and labels
ax.set_xlim(0, 100)
ax.set_ylim(0, 1)
ax.set_xticks(range(0, 101, 10))
ax.set_yticks([])  # No y-axis ticks

# Add labels
for i in range(len(percentages)):
    label_x = cumulative_percentages[i] - (width / 2) if i > 0 else width / 2
    ax.text(label_x, 0.5, f'{percentages[i]}%', ha='center', va='center', color='white', fontsize=12)

# Customize the plot
ax.set_title('Flat Pie Chart')
ax.set_xlabel('Percentage')

# Show the plot
plt.show()

And now do the same for total building contibutions.

In [ ]:
denum = nondom_building_count_run_sum + dom_building_count_run_sum + mixed_building_count_run_sum
print(f"Total = {denum}")
percentages = [
                100*dom_building_count_run_sum/denum,
                100*mixed_building_count_run_sum/denum,
                100*nondom_building_count_run_sum/denum
               ]

print(percentages)

# Colors for each segment
colors = ['blue', 'orange', 'green']

# Calculate cumulative percentages
cumulative_percentages = [sum(percentages[:i+1]) for i in range(len(percentages))]

# Create the horizontal bar plot
fig, ax = plt.subplots()

for i in range(len(percentages)):
    width = percentages[i]
    start_percentage = cumulative_percentages[i] - width if i > 0 else 0
    rect = plt.Rectangle((start_percentage, 0), width, 1, color=colors[i], edgecolor='white')
    ax.add_patch(rect)

# Set plot limits and labels
ax.set_xlim(0, 100)
ax.set_ylim(0, 1)
ax.set_xticks(range(0, 101, 10))
ax.set_yticks([])  # No y-axis ticks

# Add labels
for i in range(len(percentages)):
    label_x = cumulative_percentages[i] - (width / 2) if i > 0 else width / 2
    ax.text(label_x, 0.5, f'{percentages[i]}%', ha='center', va='center', color='white', fontsize=12)

# Customize the plot
ax.set_title('Flat Pie Chart')
ax.set_xlabel('Percentage')

# Show the plot
plt.show()

Now make a map of total energy usage over the whole place.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datamethods.plotting import plot_region

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
gdf["tot_per_floor"] = gdf["total_energy"]/(gdf["nofloors"]*10**6)
plot_region(df, colour_variable="tot_per_floor", colourmap="viridis", ax=ax, linewideth=0.1)
plt.savefig("paperfigs/mean_energy_per_floor.pdf")

Energy usage by usage type.

In [ ]:
data = {
    "dom": [],
    "nondom": [],
    "mixed": []
}
for ind, row in gdf.iterrows():
        type = row["type"]
        data[type].append(row["total_energy"]/10**3)
             

categories_to_plot = ["dom", "nondom", "mixed"]
# Create a new dictionary with selected categories
selected_data = {category: data[category] for category in categories_to_plot}

print(f"Median energy usage for domestic buildings: {np.mean(data['dom'])}")
print(f"Median energy usage for non-domestic buildings: {np.mean(data['nondom'])}")
print(f"Median energy usage for mixed buildings: {np.mean(data['mixed'])}")

# Extract category names and values
categories = list(selected_data.keys())
values = list(selected_data.values())

# Create box plots
plt.boxplot(values, labels=categories)
plt.yscale("log")
plt.tight_layout()
plt.ylabel("Total annual building energy usage (MWh)")
plt.show()